In [1]:
import keras_tuner as kt


In [2]:
import sys, os
root = os.path.abspath("..")
if root not in sys.path: sys.path.insert(0, root)

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt    
from src.models import build_mediumnet

def model_builder(hp):
    model = build_mediumnet(input_shape=(150,150,3), n_classes=3)
    lr = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model



In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from pathlib import Path

# Set paths 
data_path = Path("../data/split")  # use your actual split directory

IMG_SIZE = (150, 150)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Load datasets from directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_path / "train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_path / "val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

# Normalize pixel values to [0, 1]
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)


Found 1750 files belonging to 3 classes.
Found 219 files belonging to 3 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
tuner = kt.RandomSearch(
    model_builder,
    objective="val_accuracy",
    max_trials=8,
    executions_per_trial=1,
    directory="kt_dir",
    project_name="rps_mediumnet"
)

tuner.search_space_summary()

tuner.search(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

tuner.results_summary()

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best learning rate:", best_hp.get("learning_rate"))


Reloading Tuner from kt_dir\rps_mediumnet\tuner0.json
Search space summary
Default search space size: 1
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
Results summary
Results in kt_dir\rps_mediumnet
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
learning_rate: 0.0006395846481676142
Score: 0.949999988079071

Trial 7 summary
Hyperparameters:
learning_rate: 0.0008385828184982733
Score: 0.9363636374473572

Trial 0 summary
Hyperparameters:
learning_rate: 0.0008711452772441899
Score: 0.9318181872367859

Trial 4 summary
Hyperparameters:
learning_rate: 0.0003695396936544449
Score: 0.8999999761581421

Trial 6 summary
Hyperparameters:
learning_rate: 0.0001862289652494819
Score: 0.8590909242630005

Trial 2 summary
Hyperparameters:
learning_rate: 0.0018737848019708316
Score: 0.34090909361839294

Trial 5 summary
Hyperparameters:
learning_rate: 0.00148354745

In [5]:
#more optimized version-test----TinyNet +
import time
import tensorflow as tf
from src.models import build_tinynet
from tensorflow.keras.callbacks import EarlyStopping

# 1) Re-prepare datasets with caching and larger batch
def fast_ds(split, batch_size=64):
    ds = tf.keras.utils.image_dataset_from_directory(
        f"../data/processed/{split}",
        image_size=(150,150),
        batch_size=batch_size,
        labels="inferred",
        label_mode="int",
        shuffle=(split=="train"),
        seed=42
    )
    ds = ds.map(lambda x,y: (tf.keras.layers.Rescaling(1./255)(x), y),
                num_parallel_calls=tf.data.AUTOTUNE)
    if split=="train":
        ds = ds.cache()                              # cache in memory
    return ds.prefetch(tf.data.AUTOTUNE)

train_fast = fast_ds("train", batch_size=64)
val_fast   = fast_ds("val",   batch_size=64)
test_fast  = fast_ds("test",  batch_size=64)

# 2) Build and compile TinyNet
model = build_tinynet(input_shape=(150,150,3), n_classes=3)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 3) Train with early stopping, timing the run
es = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
start = time.time()
hist = model.fit(
    train_fast,
    epochs=10,             # fewer epochs
    validation_data=val_fast,
    callbacks=[es],
    verbose=2
)
elapsed = time.time() - start
print(f"\n⏱️  TinyNet training time: {elapsed:.1f} sec")

# 4) Evaluate on test
loss, acc = model.evaluate(test_fast, verbose=0)
print(f"🎯 TinyNet test accuracy: {acc:.4f}, loss: {loss:.4f}")


Found 1748 files belonging to 3 classes.
Found 220 files belonging to 3 classes.
Found 220 files belonging to 3 classes.
Epoch 1/10
28/28 - 11s - loss: 1.1016 - accuracy: 0.3392 - val_loss: 1.0961 - val_accuracy: 0.3909 - 11s/epoch - 381ms/step
Epoch 2/10
28/28 - 9s - loss: 1.0952 - accuracy: 0.3467 - val_loss: 1.0907 - val_accuracy: 0.4318 - 9s/epoch - 318ms/step
Epoch 3/10
28/28 - 9s - loss: 1.0873 - accuracy: 0.3982 - val_loss: 1.0762 - val_accuracy: 0.4500 - 9s/epoch - 332ms/step
Epoch 4/10
28/28 - 9s - loss: 1.0723 - accuracy: 0.4376 - val_loss: 1.0477 - val_accuracy: 0.5364 - 9s/epoch - 323ms/step
Epoch 5/10
28/28 - 9s - loss: 1.0513 - accuracy: 0.4411 - val_loss: 1.0154 - val_accuracy: 0.4818 - 9s/epoch - 331ms/step
Epoch 6/10
28/28 - 9s - loss: 1.0202 - accuracy: 0.4760 - val_loss: 0.9757 - val_accuracy: 0.5273 - 9s/epoch - 321ms/step
Epoch 7/10
28/28 - 9s - loss: 0.9834 - accuracy: 0.4908 - val_loss: 0.9485 - val_accuracy: 0.5682 - 9s/epoch - 325ms/step
Epoch 8/10
28/28 - 9s -